In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/oxford-102-flowers/102flowers.tgz
/kaggle/input/oxford-102-flowers/imagelabels.mat
/kaggle/input/mobilenet-weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5
/kaggle/input/weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5


In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import zipfile
import seaborn as sns
import tensorflow as tf
#from google.colab.patches import cv2_imshow
#from google.colab import drive
%matplotlib inline
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense

In [3]:
import tarfile         # To deal with tgz file
path='/kaggle/input/oxford-102-flowers/102flowers.tgz'
tfile = tarfile.open(path, "r:gz")
tfile.extractall('./')
tfile.close()

In [4]:
import scipy.io      #To deal with mat file
mat = scipy.io.loadmat('/kaggle/input/oxford-102-flowers/imagelabels.mat')
label=mat["labels"].tolist()
l=np.reshape(label,(-1,1))

In [5]:
directory='/kaggle/working/jpg'
files=[os.path.join(directory,f) for f in sorted(os.listdir(directory))]

In [6]:
df=pd.DataFrame(columns=["Files","labels"])        #
df["Files"]=files
df["labels"]=l
y=df["labels"]
df.head()

,Files,labels
0,/kaggle/working/jpg/image_00001.jpg,77
1,/kaggle/working/jpg/image_00002.jpg,77
2,/kaggle/working/jpg/image_00003.jpg,77
3,/kaggle/working/jpg/image_00004.jpg,77
4,/kaggle/working/jpg/image_00005.jpg,77


In [7]:
output_shape = (128,128)
images=[]
for image_path in files :
  img = Image.open(image_path)
  img_resized = np.array(img.resize(output_shape, Image.ANTIALIAS))
  images.append(img_resized)


/tmp/ipykernel_19/2848993762.py:5: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img_resized = np.array(img.resize(output_shape, Image.ANTIALIAS))


In [8]:
images[0].shape

(128, 128, 3)

In [9]:
normalized_images = []
for img in images:
    # Convert the image to float32 for normalization
    img_float = img.astype(np.float32)

    # Normalize the pixel values using cv2.NORM_MINMAX
    img_normalized = cv2.normalize(img_float, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    # Append the normalized image to the list
    normalized_images.append(img_normalized)

In [10]:
x=np.array(normalized_images)
Y=np.array(y)
X_train,X_test,y_train,y_test=train_test_split(x,Y,test_size=0.2,random_state=42,shuffle=True)
X_trainn,X_val,y_trainn,y_val=train_test_split(X_train,y_train,test_size=0.2,random_state=42,shuffle=True)
X_trainn.shape,y_trainn.shape,X_val.shape,y_val.shape,X_test.shape,y_test.shape

((5240, 128, 128, 3),
 (5240,),
 (1311, 128, 128, 3),
 (1311,),
 (1638, 128, 128, 3),
 (1638,))

In [11]:
from tensorflow.keras.applications import ResNet50
from keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
#weights="/kaggle/input/weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
from tensorflow.keras.applications import MobileNetV2

In [12]:
input_shape=(128,128,3)
weights="/kaggle/input/mobilenet-weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5"
inp = Input(shape=input_shape)
mobile_model = MobileNetV2(weights=None,include_top=False,input_tensor=inp)
mobile_model.load_weights(weights)
mobile_model.trainable = False
mobile_model.summary()

Model: "mobilenetv2_1.00_None"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 64, 64, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 64, 64, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 64, 64, 32)           0         ['bn_Conv1

In [13]:
x= mobile_model.output
x = Flatten()(x)
x = Dense(10291, activation='relu')(x)
predictions = Dense(103, activation='softmax')(x)
model = Model(inputs=inp, outputs=predictions)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 64, 64, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 64, 64, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 64, 64, 32)           0         ['bn_Conv1[0][0]']        

In [14]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(X_trainn, y_trainn, epochs=10, batch_size=20,validation_data=(X_val,y_val),callbacks=[early_stopping])

Epoch 1/10
262/262 [==============================] - 735s 3s/step - loss: 5.9903 - accuracy: 0.5149 - val_loss: 1.4422 - val_accuracy: 0.6186
Epoch 2/10
262/262 [==============================] - 695s 3s/step - loss: 0.6048 - accuracy: 0.8336 - val_loss: 1.3871 - val_accuracy: 0.6697
Epoch 3/10
262/262 [==============================] - 687s 3s/step - loss: 0.3323 - accuracy: 0.9057 - val_loss: 1.6520 - val_accuracy: 0.6850
Epoch 4/10
262/262 [==============================] - 678s 3s/step - loss: 0.2830 - accuracy: 0.9277 - val_loss: 1.9734 - val_accuracy: 0.6743
Epoch 5/10
262/262 [==============================] - 684s 3s/step - loss: 0.3340 - accuracy: 0.9237 - val_loss: 1.9785 - val_accuracy: 0.7071
Epoch 6/10
262/262 [==============================] - 680s 3s/step - loss: 0.2677 - accuracy: 0.9384 - val_loss: 2.0939 - val_accuracy: 0.6995
Epoch 7/10
262/262 [==============================] - 684s 3s/step - loss: 0.2982 - accuracy: 0.9378 - val_loss: 2.7718 - val_accuracy: 0.6690

In [15]:
predictions=model.predict(X_test)
predictions

52/52 [==============================] - 22s 386ms/step


array([[2.8065840e-11, 1.7511609e-07, 1.8897248e-08, ..., 1.7947540e-08,
        4.8887259e-06, 7.0750832e-09],
       [2.4989485e-10, 3.3706015e-06, 4.4681283e-08, ..., 5.5213377e-04,
        5.8305767e-07, 9.6606379e-07],
       [2.9206134e-11, 4.3036555e-10, 6.5665324e-07, ..., 1.4350553e-07,
        1.5109204e-07, 4.6464074e-10],
       ...,
       [9.3824717e-09, 1.5144846e-04, 1.7556260e-05, ..., 6.8787904e-06,
        7.7519191e-09, 2.0056007e-08],
       [5.2721125e-16, 5.2692952e-13, 2.6716326e-17, ..., 1.1821329e-11,
        9.3347649e-11, 1.2772204e-11],
       [3.3032171e-11, 3.2240962e-07, 2.8403113e-06, ..., 1.2131304e-09,
        3.2740766e-06, 2.1620262e-07]], dtype=float32)

In [16]:
predictions=np.argmax(predictions,axis=1)
predictions

array([84, 18, 99, ..., 53, 30, 40])

In [17]:
from sklearn.metrics import accuracy_score,classification_report
accuracy_score(y_test,predictions)

0.6758241758241759